# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:

cur.execute("""
    SELECT
         table_schema
        ,table_name
        ,column_name
        ,ordinal_position
        ,is_nullable
        ,data_type
        
    FROM
        information_schema.columns
        
    WHERE
        table_schema NOT IN ('pg_catalog','information_schema')
        AND
        ordinal_position = 1
"""
)

row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

('public', 'dim_users', 'user_id', 1, 'NO', 'integer')
('public', 'dim_songs', 'song_id', 1, 'NO', 'text')
('public', 'dim_artists', 'artist_id', 1, 'NO', 'text')
('public', 'dim_time', 'start_time', 1, 'NO', 'timestamp with time zone')
('public', 'fact_songplays', 'songplay_id', 1, 'NO', 'integer')


In [4]:

for (root,dirs,files) in os.walk('data/song_data',topdown=True):
    print(root)
    print(dirs)
    print(files)

data/song_data
['A', '.ipynb_checkpoints']
[]
data/song_data/A
['B', 'A', '.ipynb_checkpoints']
[]
data/song_data/A/B
['C', 'B', 'A', '.ipynb_checkpoints']
[]
data/song_data/A/B/C
['.ipynb_checkpoints']
['TRABCRU128F423F449.json', 'TRABCTK128F934B224.json', 'TRABCUQ128E0783E2B.json', 'TRABCIX128F4265903.json', 'TRABCYE128F934CE1D.json', 'TRABCEC128F426456E.json', 'TRABCXB128F4286BD3.json', 'TRABCAJ12903CDFCC2.json', 'TRABCPZ128F4275C32.json', 'TRABCKL128F423A778.json', 'TRABCEI128F424C983.json', 'TRABCFL128F149BB0D.json']
data/song_data/A/B/C/.ipynb_checkpoints
[]
['TRABCRU128F423F449-checkpoint.json']
data/song_data/A/B/B
[]
['TRABBBV128F42967D7.json', 'TRABBLU128F93349CF.json', 'TRABBXU128F92FEF48.json', 'TRABBAM128F429D223.json', 'TRABBJE12903CDB442.json', 'TRABBNP128F932546F.json', 'TRABBOP128F931B50D.json', 'TRABBZN12903CD9297.json', 'TRABBOR128F4286200.json', 'TRABBTA128F933D304.json', 'TRABBKX128F4285205.json', 'TRABBVJ128F92F7EAA.json']
data/song_data/A/B/A
[]
['TRABACN128F425B

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
filepath = 'data/song_data'

song_files = get_files(filepath)

song_files[:3]

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json']

In [6]:

# the entire JSON object is ingested in string data type
df = pd.read_json( 
     song_files[0]
    ,typ='frame'
    ,lines=True
    ) 
    #.astype(object)

df.head()

# df.dtypes

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [7]:

float_cols_default_value = 9999999.99

string_cols_default_value = 'Unknown'

float_cols = ['artist_latitude','artist_longitude','duration']

integer_cols = ['year']

string_attribute_cols = [ 'artist_id'
                         ,'artist_location'
                         ,'artist_name'
                         ,'song_id'
                         ,'title'
                        ]

df[float_cols] = df[float_cols].fillna(value=float_cols_default_value)

# insere valores-padrão para evitar dimensões com atributos nulos
df[string_attribute_cols] = df[string_attribute_cols].fillna(value='Unknown').replace(to_replace='',value='Unknown')

df = df.astype(object)

In [8]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,1e+07,Unknown,1e+07,Marc Shaiman,149.864,1,SOINLJW12A8C13314C,City Slickers,2008


In [10]:
# retrie only the first row of data, from five specific columns:
df.loc[0,['song_id','title','artist_id','year','duration']].values.tolist()

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
# retrie only the first row of data, from five specific columns:
song_data = df.loc[0,['song_id','title','artist_id','year','duration']].values.tolist()

song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

In [12]:
song_table_insert = """
    TRUNCATE TABLE
        public.dim_songs
    CASCADE
    ;

    INSERT INTO
        public.dim_songs(
             song_id
            ,title
            ,artist_id
            ,year
            ,duration
        )

    VALUES
        (
             %s
            ,%s
            ,%s
            ,%s
            ,%s
        )
    ;
"""

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [14]:
conn.rollback()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
artist_data = df.loc[0,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', 'Unknown', 9999999.99, 9999999.99]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
conn.rollback()
# conn.close()

In [15]:
artist_table_insert = """
    TRUNCATE TABLE
        public.dim_artists
    CASCADE
    ;
    
    INSERT INTO
        public.dim_artists(
             artist_id
            ,name
            ,location
            ,latitude
            ,longitude
        )
        
    VALUES(
         %s
        ,COALESCE(REPLACE(%s,'',NULL),'Unknown')
        ,COALESCE(REPLACE(%s,'',NULL),'Unknown')
        ,COALESCE(%s,999.99)
        ,COALESCE(%s,999.99)
    )
"""


cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
filepath = 'data/log_data'

log_files = get_files(filepath)

In [17]:
log_files[0]

'/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [18]:
df = pd.read_json( 
         log_files[0]
        ,typ='frame'
        ,lines=True
     )

# df = df.head(1)

df.head()
# df.dtypes

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [19]:

log_string_attributes = [ 'artist'
                         ,'auth'
                         ,'firstName'
                         ,'gender'
                         ,'lastName'
                         ,'level'
                         ,'location'
                         ,'method'
                         ,'page'
                         ,'song'
                         ,'userAgent'
                         ,'userId'
                        ]

log_float_attributes = [ 'length'
                        ,'registration'
                       ]

log_int_attributes = [ 'itemInSession'
                      ,'sessionId'
                      ,'status'
                      ,'ts'
                     ]

# boolean indexing is used
next_song_criteria = df['page'] == 'NextSong'

df['ts'] = pd.to_datetime(df['ts'],unit='ms')

df['hour'] = df['ts'].dt.hour

df.loc[next_song_criteria].head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,hour
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,0


In [20]:

targe_time_attributes = [ 'start_time'
                         ,'hour'
                         ,'day'
                         ,'week'
                         ,'month'
                         ,'year'
                         ,'weekday'
                        ]


time_components_list = []

time_components_list.append(df['ts'].dt.strftime('%Y%m%d %H:%M:%S.%f').values[0])
    
time_components_list.append(df['ts'].dt.hour.values[0])

time_components_list.append(df['ts'].dt.day.values[0])

time_components_list.append(df['ts'].dt.week.values[0])

time_components_list.append(df['ts'].dt.month.values[0])

time_components_list.append(df['ts'].dt.year.values[0])

time_components_list.append(df['ts'].dt.weekday_name.values[0])
    


# df['ts'].dt.time.values[0].apply(lambda x: x.strftime('%H%M%S'))

# df['ts'].dt.time.values[0]

# df['ts'].dt.strftime('%H:%M:%S').values[0]

time_components_list




['20181129 00:00:57.796000', 0, 29, 48, 11, 2018, 'Thursday']

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
# df = 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,hour
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,0
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24,0
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24,0
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,0
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24,0


In [22]:
t = 
t.head()

SyntaxError: invalid syntax (<ipython-input-22-694dba48915f>, line 1)

In [23]:
time_data = ()
column_labels = ( 'start_time'
                 ,'hour'
                 ,'day'
                 ,'week'
                 ,'month'
                 ,'year'
                 ,'weekday'
                )

In [24]:
time_df = pd.DataFrame( data=[time_components_list]
                       ,columns=column_labels
                      )
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,20181129 00:00:57.796000,0,29,48,11,2018,Thursday


In [25]:

time_table_insert = """
    TRUNCATE TABLE
        public.dim_time
    CASCADE
    ;
    
    INSERT INTO
        public.dim_time(
             start_time
            ,hour
            ,day
            ,week
            ,month
            ,year
            ,weekday
        )
        
    VALUES (
         %s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
conn.rollback()

In [26]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [27]:

target_user_attributes = [ 'userId'
                          ,'firstName'
                          ,'lastName'
                          ,'gender'
                          ,'level'
                         ]

user_df = df.loc[0,target_user_attributes].to_frame().T

user_df

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid


In [28]:

user_table_insert = """
    TRUNCATE TABLE
        public.dim_users
    CASCADE
    ;
    
    INSERT INTO
        public.dim_users(
             user_id
            ,first_name
            ,last_name
            ,gender
            ,level
        )
        
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [29]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [30]:

song_select = """
    SELECT
         dim_songs.song_id
        ,dim_songs.artist_id
        
    FROM
        public.dim_songs AS dim_songs
        
    LEFT JOIN
        public.dim_artists AS dim_artists
        
    ON
        dim_songs.artist_id
        =
        dim_artists.artist_id
        
    WHERE
        dim_songs.title = (%s)
        AND
        dim_artists.name = (%s)
        AND
        dim_songs.duration = (%s)
        
    ;
"""

cur.execute(song_select)

row = cur.fetchone()

pd.read_sql( song_select
            ,conn
           )

ProgrammingError: syntax error at or near "%"
LINE 19:         dim_songs.title = (%s)
                                    ^


In [33]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    print(results)

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

None
None


IntegrityError: insert or update on table "fact_songplays" violates foreign key constraint "fact_songplays_start_time_fkey"
DETAIL:  Key (start_time)=(2018-11-29 00:01:30.796+00) is not present in table "dim_time".


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [34]:
# conn.rollback()

conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.